# Prompt Optimization with Evidently: Bookings Query Classifier
In this tutorial, we'll optimize a prompt for classifying different types of customer service queries (like Booking, Payment, or Technical issues) using an LLM classifier.

## What you'll learn:
- How to load a dataset for LLM classification
- How to define a multiclass classification prompt
- How to run prompt optimization with Evidently
- How to retrieve the best performing prompt

In [ ]:
# Install packages if needed
# !pip install evidently openai pandas

In [ ]:
import pandas as pd

from evidently import Dataset, DataDefinition, LLMClassification
from evidently.descriptors import LLMEval
from evidently.llm.templates import MulticlassClassificationPromptTemplate
from evidently.llm.optimization import PromptOptimizer

## Load Your Dataset

In [ ]:
data = pd.read_csv("../datasets/bookings.csv")
data.head()

## Define Data Structure for Evidently

In [ ]:
dd = DataDefinition(
    text_columns=["query"],
    categorical_columns=["label"],
    llm=LLMClassification(input="query", target="label")
)

In [ ]:
dataset = Dataset.from_pandas(data, data_definition=dd)

## Define a Multiclass Prompt and LLM Judge

In [ ]:
bad_prompt = "Classify inqueries by categories"

t = MulticlassClassificationPromptTemplate(
    pre_messages=[("system", "You are classifying user queries.")],
    criteria=bad_prompt,
    category_criteria={
        "Booking": "bookings",
        "Technical": "technical questions",
        "Policy": "questions about policies",
        "Payment": "payment questions",
        "Escalation": "escalation requests"
    },
    uncertainty="unknown",
    include_reasoning=True,
)

judge = LLMEval(
    alias="bookings",
    provider="openai",
    model="gpt-4o-mini",
    column_name="query",
    template=t
)

## Run the Prompt Optimizer

In [ ]:
optimizer = PromptOptimizer("bookings_example", strategy="feedback", verbose=True)
optimizer.set_input_dataset(dataset)
await optimizer.arun(judge, "accuracy", repetitions=5)
# sync version
# optimizer.run(judge, "accuracy")

## View the Best Optimized Prompt

In [ ]:
print(optimizer.best_prompt())

In [ ]:
optimizer.print_stats()